In [1]:
%matplotlib widget

import sys  
sys.path.insert(0, '/home/cbisot/pycode/MscThesis/')
import pandas as pd
from sample.util import get_dates_datetime, get_dirname, get_plate_number, get_postion_number,get_begin_index
import ast
from sample.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
from sample.pipeline.functions.node_id import orient
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from sample.pipeline.functions.extract_graph import from_sparse_to_graph, generate_nx_graph, sparse_to_doc
from skimage.feature import hessian_matrix_det
from sample.pipeline.functions.experiment_class_surf import Experiment, Edge, Node, Hyphae, plot_raw_plus
from sample.paths.directory import run_parallel, find_state, directory_scratch, directory_project
from sample.notebooks.analysis.util import *
from scipy import stats
from scipy.ndimage.filters import uniform_filter1d
from statsmodels.stats import weightstats as stests
from sample.pipeline.functions.hyphae_id_surf import get_pixel_growth_and_new_children
from collections import Counter
from IPython.display import clear_output
from sample.notebooks.analysis.data_info import *


In [2]:
exp = get_exp((39,269,329),directory_project)


begin = 2020-11-23 15:01:00 
  end = 2020-11-26 03:02:00


In [3]:
def criter(max_growth,length):
    return(a*length+b*max_growth>=2.5 and max_growth>=50)

In [4]:
RH, BAS, max_speeds, total_growths, widths, lengths, branch_frequ,select_hyph = get_rh_bas(exp,criter)

failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure 

In [14]:
np.mean([speed for speed in max_speeds if speed>25])

116.98713915375436

In [5]:
hyph_anas_tip_hyph = [hyphat for hyphat in exp.hyphaes if len(hyphat.ts)>=2 and hyphat.end.degree(hyphat.ts[-1])>=3 and hyphat.end.degree(hyphat.ts[-2])>=3]


In [6]:
branches = {}
lengths = {}
bd = []
rh_ends = [rh.end for rh in RH]
daughters = {}
begins = {}
for rh in RH:
    tp1 = rh.ts[-1]
    t = rh.ts[0]
    pixels, nodes = get_pixel_growth_and_new_children(rh, t, tp1)
    curv_length = np.sum([get_length_um(seg) for seg in pixels])
    branch = []
    daughter_list=[]
    begins[rh] = nodes[1]
    for node in nodes[1:-1]:
        if node not in [hyph.end for hyph in hyph_anas_tip_hyph]:
            node_obj  = node
#             t0 = node_obj.ts()[0]
#             neighbours = node_obj.neighbours(t0)
#             if t0 in rh.ts:
#                 nodes_hyph,edge = rh.get_nodes_within(t0)
#                 for neighbour in neighbours:
#                     if neighbour.label not in nodes_hyph:
#                         daughter_list.append(neighbour)
#                         if neighbour in rh_ends and neighbour!=rh.end:
            branch.append(node)    
    bd.append(len(branch)/curv_length)
    branches[rh] = branch
    daughters[rh] = daughter_list
    lengths[rh] = curv_length

failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge


In [7]:
np.mean(bd)

0.00170557685979303

In [17]:
1/np.mean(bd)*3/np.log(2)/24

105.7336578387937

In [20]:
2000*(1.73+0.8)/np.log(2)/24

304.16820445408985

In [15]:
1/np.mean(bd)

586.3118945699985

In [86]:
rh = choice(RH)
plt.close('all')
plot_raw_plus(exp,rh.ts[-1],branches[rh]+[rh.end.label,rh.root.label]+[hyph.label for hyph in daughters[rh]]+[begins[rh].label])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [84]:
rh.end.ts(),lengths[rh],branches[rh],rh

([36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60],
 239.33367708738464,
 [],
 Hyphae(5551,1746))

In [49]:
len(RH)

230

In [13]:
neighbours

[Node(891), Node(3985), Node(3)]

In [42]:
plt.close('all')
fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.hist(bd,30)

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([14.,  2., 10.,  6., 15., 21., 25., 16., 12.,  9.,  7.,  6.,  3.,
         3.,  0.,  0.,  3.,  1.,  1.,  1.,  1.,  0.,  0.,  1.,  0.,  0.,
         1.,  0.,  1.,  1.]),
 array([0.        , 0.00021597, 0.00043195, 0.00064792, 0.00086389,
        0.00107986, 0.00129584, 0.00151181, 0.00172778, 0.00194375,
        0.00215973, 0.0023757 , 0.00259167, 0.00280765, 0.00302362,
        0.00323959, 0.00345556, 0.00367154, 0.00388751, 0.00410348,
        0.00431946, 0.00453543, 0.0047514 , 0.00496737, 0.00518335,
        0.00539932, 0.00561529, 0.00583126, 0.00604724, 0.00626321,
        0.00647918]),
 <a list of 30 Patch objects>)

In [44]:
1/np.mean(bd)

639.5821582781915

In [54]:
Node(1735,exp).show_source_image(14,14)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [49]:
Node(237,exp).neighbours(rh.ts[-1])

[Node(2019), Node(8511), Node(7466)]

In [28]:
Node(16134,exp).show_source_image(rh.ts[-1],rh.ts[-1])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [30]:
edge = Edge(Node(16134,exp),Node(16141,exp),exp)

In [31]:
[get_length_um_edge(edge,t) for t in edge.ts()]

[65.71399411647921]

In [16]:
Node(5857,exp).neighbours(rh.ts[-1])

[Node(4432), Node(1612), Node(6735)]

In [5]:
data = {}
lapses = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]
curv_growths = []
straight_distances = []
lapses_time = []
for lapse in lapses:
    data[lapse]=[]
    for rh in RH:
        index = 0
        for index in range(len(rh.ts)-lapse):
            t = rh.ts[index]
            tp1 = rh.ts[index + lapse]
            pos = rh.end.pos(t)
            pos_tp1 = rh.end.pos(tp1)
            pixels, nodes = get_pixel_growth_and_new_children(rh, t, tp1)
            count=len(nodes[1:-1]) 
            lapse_time = get_time(exp, t, tp1)
            curv_growth = np.sum([get_length_um(seg) for seg in pixels])
            if curv_growth/lapse_time>25:
                straight_distance = np.linalg.norm(pos-pos_tp1)
                inv_tortuosity = (straight_distance*1.725)/curv_growth
                data[lapse].append((curv_growth,straight_distance))
                curv_growths.append(curv_growth)
                straight_distances.append(straight_distance)
                lapses_time.append(lapse_time)

In [36]:
fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.scatter(curv_growths,np.array(straight_distances)*1.725/np.array(curv_growths))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
data = pd.DataFrame(list(zip(curv_growths,straight_distances)),columns = ['curv_growth','straight_distance'])
data['straight'] = data['straight_distance']*1.725/data['curv_growth']
data['straight_scaled'] = data['straight_distance']*1.725/np.sqrt(data['curv_growth'])
data['time'] = lapses_time
data['straight_time'] = data['straight']/data['time']
data['speed'] = data['curv_growth']/data['time']

In [7]:
np.max(data['time'])

21.0

In [8]:
correct = data.loc[(data['straight']<1)& (data['speed']>150)& (data['speed']<400)]

In [10]:
plt.close('all')
abcisse = 'curv_growth'
ordinate = 'straight'
tab = correct
baits_sort = tab.sort_values(abcisse)
N=3000
moving_av = baits_sort.rolling(N,min_periods=N//2).mean()
moving_std = baits_sort.rolling(N,min_periods=N//2).std()

fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.set_ylabel('straightness')

ax.set_xlabel('curvilinear length ($\mu m$)')
# ax.set_xlim(-0.5,0.5)
ax.set_ylim(0.96,1)
# ax.set_xlim(-190,190)
# slope, intercept, r_value, p_value, std_err = stats.linregress(densities_sort,np.abs(curvatures_sort))

ax.scatter(correct[abcisse],correct[ordinate],alpha=0.3,c = correct['time'])
ax.plot(moving_av[abcisse],moving_av[ordinate],color='green',label = 'moving average')
ax.plot(moving_av[abcisse],(moving_av[ordinate]+moving_std[ordinate]/np.sqrt(N)),color='red',label = 'std')
ax.plot(moving_av[abcisse],(moving_av[ordinate]-moving_std[ordinate]/np.sqrt(N)),color='red',label = 'std')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
plt.close('all')
abcisse = 'speed'
ordinate = 'straight'
tab = correct
baits_sort = tab.sort_values(abcisse)
N=1600
moving_av = baits_sort.rolling(N,min_periods=N//2).mean()
moving_std = baits_sort.rolling(N,min_periods=N//2).std()

fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.set_xlabel('inverse density ($mm.mm^{-2}$)')

ax.set_ylabel('normalised scalar product speed.density gradient')
# ax.set_xlim(-0.5,0.5)
# ax.set_ylim(-0.5,0.5)
# ax.set_xlim(-190,190)
# slope, intercept, r_value, p_value, std_err = stats.linregress(densities_sort,np.abs(curvatures_sort))

ax.scatter(correct[abcisse],correct[ordinate],alpha=0.3,c = correct['time'])
ax.plot(moving_av[abcisse],moving_av[ordinate],color='green',label = 'moving average')
ax.plot(moving_av[abcisse],(moving_av[ordinate]+moving_std[ordinate]/np.sqrt(N)),color='red',label = 'std')
ax.plot(moving_av[abcisse],(moving_av[ordinate]-moving_std[ordinate]/np.sqrt(N)),color='red',label = 'std')

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/cbook/__init__.py:1377: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  x[:, None]
/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/axes/_base.py:237: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  x = x[:, np.newaxis]
/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/axes/_base.py:239: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  y = y[:, np.newaxis]


In [16]:
tab

,curv_growth,straight_distance,straight,straight_scaled,time,straight_time,speed
0,261.436548,151.029798,0.996519,16.112710,1.000000,0.996519,261.436548
1,264.731835,153.323188,0.999058,16.255258,1.000000,0.999058,264.731835
2,271.695566,157.257750,0.998432,16.457349,0.983333,1.015355,276.300575
3,246.001090,142.003521,0.995752,15.617794,1.000000,0.995752,246.001090
7,248.937756,143.694120,0.995720,15.710236,1.000000,0.995720,248.937756
...,...,...,...,...,...,...,...
26730,3567.965315,2043.160542,0.987804,59.003976,16.000000,0.061738,222.997832
26734,3801.541572,2181.430036,0.989853,61.031004,16.000000,0.061866,237.596348
26735,3975.449970,2236.633184,0.970505,61.191446,16.000000,0.060657,248.465623
26740,3011.331538,1731.276119,0.991738,54.422207,16.000000,0.061984,188.208221


In [75]:
roots_new_branch = []
total_growth = 0
counts =[]
speed_ratios = []
lapses = []
roots = [rh.root for rh in RH]
lag = 1 
for lapse in [1]:
    for rh in RH:
        index = 0
        while index < (len(rh.ts)):
            if index+lapse+lag<len(rh.ts):
                t = rh.ts[index]
                tp1 = rh.ts[index + lapse]
                pixels, nodes = get_pixel_growth_and_new_children(rh, t, tp1)
                count=len(nodes[1:-1]) 
                if count==0:
                    t = rh.ts[index]
                    tp1 = rh.ts[index + 1]
                    pixels, nodes = get_pixel_growth_and_new_children(rh, t, tp1)
                    lapse_time = get_time(exp, t, tp1)
                    speed_init = np.sum([get_length_um(seg) for seg in pixels])/lapse_time
                    t0 = rh.ts[index+lapse]
                    t0p1 = rh.ts[index + lapse+lag]
                    pixels, nodes = get_pixel_growth_and_new_children(rh, t0, t0p1)
                    lapse_time = get_time(exp, t0, t0p1)
                    speed_after = np.sum([get_length_um(seg) for seg in pixels])/lapse_time
                    if speed_after>100 and speed_init>100:
                        counts.append(count)
                        speed_ratios.append(speed_after/speed_init)
                    index+= lapse
            index +=1

In [77]:
roots_new_branch = []
total_growth = 0
counts =[]
speed_ratios2 = []
lapses = []
roots = [rh.root for rh in RH]
lag = 1
for lapse in [1]:
    for rh in RH:
        index = 0
        while index < (len(rh.ts)):
            if index+lapse+lag<len(rh.ts):
                t = rh.ts[index]
                tp1 = rh.ts[index + lapse]
                pixels, nodes = get_pixel_growth_and_new_children(rh, t, tp1)
                count=len(nodes[1:-1]) 
                if count==1:
                    t = rh.ts[index]
                    tp1 = rh.ts[index + 1]
                    pixels, nodes = get_pixel_growth_and_new_children(rh, t, tp1)
                    lapse_time = get_time(exp, t, tp1)
                    speed_init = np.sum([get_length_um(seg) for seg in pixels])/lapse_time
                    t0 = rh.ts[index+lapse]
                    t0p1 = rh.ts[index + lapse+lag]
                    pixels, nodes = get_pixel_growth_and_new_children(rh, t0, t0p1)
                    lapse_time = get_time(exp, t0, t0p1)
                    speed_after = np.sum([get_length_um(seg) for seg in pixels])/lapse_time
                    if speed_after>100 and speed_init>100:
                        counts.append(count)
                        speed_ratios2.append(speed_after/speed_init)
                    index+= lapse
            index +=1

In [78]:
fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.hist(speed_ratios,20)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([ 5.,  8.,  4.,  7.,  9.,  8., 11., 27., 64., 61., 30., 21., 12.,
         4.,  0.,  3.,  1.,  2.,  0.,  1.]),
 array([0.36922634, 0.44160127, 0.51397621, 0.58635115, 0.65872609,
        0.73110103, 0.80347597, 0.87585091, 0.94822585, 1.02060079,
        1.09297573, 1.16535067, 1.23772561, 1.31010055, 1.38247549,
        1.45485043, 1.52722537, 1.59960031, 1.67197525, 1.74435019,
        1.81672513]),
 <a list of 20 Patch objects>)

In [79]:
len(speed_ratios),np.median(speed_ratios),np.std(speed_ratios)/np.sqrt(len(speed_ratios))

(278, 1.0178450293319536, 0.012873430520369749)

In [80]:
len(speed_ratios2),np.median(speed_ratios2),np.std(speed_ratios2)/np.sqrt(len(speed_ratios2))

(217, 1.012384783370696, 0.015038422391621915)

In [81]:


fig = plt.figure()
bins = np.linspace(0.4, 1.6, 40)
ax = fig.add_subplot(111)
ax.hist(speed_ratios,bins,alpha=0.5,label='no branching',density=True)
ax.hist(speed_ratios2,bins,alpha=0.5,label='branching',density=True)

ax.set_xlabel(r'new_speed/former_speedr ratio')
plt.legend(loc='upper right')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [59]:
fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.hist(speed_ratios2,20)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([ 1.,  2.,  5.,  3.,  9.,  8.,  4., 10., 10., 27., 54., 35., 28.,
        17.,  8.,  6.,  4.,  1.,  2.,  1.]),
 array([0.41220617, 0.46804189, 0.52387762, 0.57971334, 0.63554906,
        0.69138479, 0.74722051, 0.80305623, 0.85889196, 0.91472768,
        0.9705634 , 1.02639913, 1.08223485, 1.13807057, 1.1939063 ,
        1.24974202, 1.30557774, 1.36141347, 1.41724919, 1.47308491,
        1.52892064]),
 <a list of 20 Patch objects>)

In [3]:
exp.load_compressed_skel()

In [105]:
directory = directory_project

plate_number = 94
plate = get_postion_number(plate_number)
begin = 269
end = 329
dates_datetime = get_dates_datetime(directory,plate)
dates = dates_datetime[begin:end+1]
exp2 = Experiment(plate,directory)
exp2.load(dates) #for method 2

In [3]:
def get_time(dates_datetimes,i,j):
    seconds = (dates_datetimes[j]-dates_datetimes[i]).total_seconds()
    return(seconds/3600)

In [3]:
inst = (39,269,329)
directory = directory_project
plate,begin,end = inst
dates_datetime = get_dates_datetime(directory,plate)
dates_datetime_chosen=dates_datetime[begin:end+1]
dates = [f'{0 if date.month<10 else ""}{date.month}{0 if date.day<10 else ""}{date.day}_{0 if date.hour<10 else ""}{date.hour}{0 if date.minute<10 else ""}{date.minute}' for date in dates_datetime_chosen]
skels = []
ims = []
kernel = np.ones((5,5),np.uint8)
itera = 1
for date in dates:
    directory_name=f'2020{date}_Plate{0 if plate<10 else ""}{plate}'
    path_snap=directory+directory_name
    skel_info = read_mat(path_snap+'/Analysis/skeleton_pruned_compressed.mat')
    skel = skel_info['skeleton']
    skels.append(cv2.dilate(skel.astype(np.uint8),kernel,iterations = itera))
    im = read_mat(path_snap+'/Analysis/raw_image.mat')['raw']
    ims.append(im)

In [13]:
i=10
boundaries=(2800,3700,3100,4200)

to_plot = ims[i][boundaries[0]:boundaries[1],boundaries[2]:boundaries[3]]
plot_t_tp1([], [], None, None, to_plot*0, to_plot,time=f't = {int(get_time(dates_datetime_chosen,0,i))}h')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
start=0
finish = end-begin
boundaries=(2800,3700,3100,4200)
for i in range(start,finish):
    to_plot = ims[i][boundaries[0]:boundaries[1],boundaries[2]:boundaries[3]]
    plt.close('all')
    clear_output(wait=True)
    plot_t_tp1([], [], None, None, to_plot*0, to_plot, save=f'/home/cbisot/pycode/MscThesis/sample/notebooks/plotting/Figure/im{i}',time=f't = {int(get_time(dates_datetime_chosen,0,i))}h')
img_array = []
for t in range(start,finish):
    img = cv2.imread(f'/home/cbisot/pycode/MscThesis/sample/notebooks/plotting/Figure/im{t}.png')
    img_array.append(img)
plate_label = plate_number[inst] 
imageio.mimsave(f'/home/cbisot/pycode/MscThesis/sample/notebooks/plotting/Figure/movie_close_up{plate_label}temp_ {begin}_{end}_{dates[start]}_{dates[finish]}.gif', img_array,duration = 1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
plate_label = plate_number[inst] 
imageio.mimsave(f'/home/cbisot/pycode/MscThesis/sample/notebooks/plotting/Figure/movie_close_up{plate_label}temp_ {begin}_{end}_{dates[start]}_{dates[finish]}.gif', img_array,duration = 1)

In [4]:
problems = []
for rh in RH:
    roots = []
    hyph = rh
    tip  =hyph.end
    for t in tip.ts():
        #             print(t,tip)
        if tip.degree(t) == 1:
            root, edges, nodes = hyph.get_edges(t, 200)
            roots.append(root)
    occurence_count = Counter(roots)
    if occurence_count.most_common(2)[0][0]!=rh.root:
        problems.append(rh)

In [5]:
len(problems),len(RH)

(17, 175)

In [6]:
rh = choice(problems)
rh

Hyphae(8135,8193)

In [9]:
rh.ts,rh.end.ts(),occurence_count.most_common(2)[0][0].ts()

([21, 22], [21, 22, 23, 24, 25], [22, 23, 24, 25])

In [8]:
roots = []
hyph = rh
tip  =hyph.end
for t in rh.end.ts():
    #             print(t,tip)
    if tip.degree(t) == 1:
        root, edges, nodes = hyph.get_edges(t, 200)
        roots.append(root)
occurence_count = Counter(roots)
occurence_count

Counter({Node(8138): 1, Node(9062): 1, Node(8999): 2})

In [11]:
plt.close('all')
plot_raw_plus(exp,rh.ts[0]+1,[rh.end.label,rh.root.label]+[occurence_count.most_common(2)[0][0].label])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [32]:
exp.plot([rh.ts[0]],[[rh.end.label,rh.root.label]+[occurence_count.most_common(2)[0][0].label]])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
rh = choice(select)
rh.mother,rh,rh.ts

NameError: name 'select' is not defined

In [7]:
roots = []
hyph = rh.mother[0]
tip  =Node(hyph.end.label,exp2)
hypha = Hyphae(tip)
for t in tip.ts():
    #             print(t,tip)
    if tip.degree(t) == 1:
        root, edges, nodes = hypha.get_edges(t, 200)
        roots.append(root)
occurence_count = Counter(roots)
occurence_count

NameError: name 'exp2' is not defined

In [8]:
roots = []
hyph = rh.mother[0]
tip  =hyph.end
for t in tip.ts():
    #             print(t,tip)
    if tip.degree(t) == 1:
        root, edges, nodes = hyph.get_edges(t, 200)
        roots.append(root)
occurence_count = Counter(roots)
occurence_count

Counter({Node(5243): 2, Node(4144): 1})

In [6]:
len(problems)

48

In [5]:
len(problems)

0

In [3]:
from sample.pipeline.functions.hyphae_id_surf import (
    clean_and_relabel,
    get_mother,
    save_hyphaes,
    resolve_ambiguity_two_ends,
    clean_obvious_fake_tips,
    width_based_cleaning
)
exp_clean = clean_and_relabel(exp)
RH, BAS, max_speeds, total_growths, widths, lengths, branch_frequ,select_hyph = get_rh_bas(exp_clean,criter)

0.0
0.055233360950013806
0.11046672190002761
0.16570008285004142
0.22093344380005522
0.27616680475006905
0.33140016570008285
0.38663352665009665
0.44186688760011045
0.4971002485501243
0.5523336095001381
0.6075669704501518
0.6628003314001657
0.7180336923501796
0.7732670533001933
0.8285004142502072
0.8837337752002209
0.9389671361502347
0.9942004971002486
Detected problems during hyphae detection, 83 hyphaes have inconsistent root over time
removing  210  nodes out of 8985 because they appear only in one timestep and are not within an identified hypha
[45.888564]
[1.136003, 37.595403]
[57.012361]
[61.589241]
[69.797028]
[72.570175]
[79.39028]
[82.292056]
[88.018915]
[93.215148]
[94.431121]
[102.698136]
[107.651592]
[114.85416]
[124.033512]
[135.147506]
[142.897725]
[149.376738]
[152.434632]
[161.84289]
[173.575468]
[188.503416]
[191.833005]
[206.577216]
[220.747472]
[239.920172]
[264.566085]
[273.541532]
[297.374064]
[318.481112]
[339.248447]
[361.865686]
[383.728288]
[407.257764]
[444.27

In [104]:
occurence_count

Counter({Node(4338): 26, Node(12705): 1})

In [99]:
hyph.root.ts()

[47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60]

In [109]:
occurence_count.most_common(2)[0][0]

Node(4338)

In [112]:
Node(9144,exp).degree(50)

3

In [5]:
roots_new_branch = []
total_growth = 0
counts =[]
total_growths = []
lapses = []
for lapse in [8,7,6,5,4,3,2,1]:
    for rh in RH:
        for index in range(len(rh.ts)):
                if index+lapse<len(rh.ts):
                    t = rh.ts[index]
                    tp1 = rh.ts[index + lapse]
                    pixels, nodes = get_pixel_growth_and_new_children(rh, t, tp1)
                    count=len(nodes[1:-1]) 
                    total_growth = np.sum([get_length_um(seg) for seg in pixels])
                    counts.append(count)
                    total_growths.append(total_growth)
                    lapses.append(lapse)


In [28]:
plt.close('all')
plate = 94
data = {'y':  counts,
        'x': lapses,
       'z' : total_growths}
df = pd.DataFrame (data)
df['frequ']=df['y']/df['z']
abcisse = 'z'
ordinate = 'frequ'
tab = df
baits_sort = tab.sort_values(abcisse)
N=1500
moving_av = baits_sort.rolling(N,min_periods=N//2).mean()
moving_std = baits_sort.rolling(N,min_periods=N//2).std()

fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.set_xlabel('alpha (°)')
ax.set_ylabel('angle between speed vector and density gradient(°)')
# ax.set_xlim(-40,40)
# ax.set_ylim(-60,60)
# ax.set_xlim(-190,190)
# slope, intercept, r_value, p_value, std_err = stats.linregress(densities_sort,np.abs(curvatures_sort))
ax.scatter(tab[abcisse],tab[ordinate],label=plate,alpha=0.3)
ax.plot(moving_av[abcisse],moving_av[ordinate],color='green',label = 'moving average')
ax.plot(moving_av[abcisse],(moving_av[ordinate]+moving_std[ordinate]/np.sqrt(N)),color='red',label = 'std')
ax.plot(moving_av[abcisse],(moving_av[ordinate]-moving_std[ordinate]/np.sqrt(N)),color='red',label = 'std')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [29]:
values = []
for lapse in [1,2,3,4,5,6,7,8]:
    aft  = df.loc[df['x']==lapse]
    value = np.mean(aft['y']/aft['z'])
    values.append(value)


In [30]:
fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.plot([1,2,3,4,5,6,7,8],values)

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …